# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

#gmaps.configure(api_key="")
gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../output_data/cities.csv")

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
humidity = cities_df["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities_df[["Lat", "Lng"]]

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False
                                 ,max_intensity=maxhumidity, point_radius=2)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
by_city_df = cities_df.loc[(cities_df['Max Temp'] > 70) & (cities_df["Max Temp"] < 80) & (cities_df["Cloudiness"] == 0) &(cities_df["Wind Speed"] <10),:]
by_city_df = by_city_df.dropna(how='any')
by_city_df.reset_index(inplace=True)
del by_city_df['index']

In [6]:
by_city_df.sort_values("City", inplace = True)
by_city_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
6,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17
1,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
5,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
2,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
4,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
0,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
3,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#For 5000 meters, I made an assumption that it is 5000 sq meters and calculated the radius which is 40.
hotel_df = by_city_df.copy()

hotellist = []
for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]['Lat']
    lng = hotel_df.loc[i]['Lng']
    
    params = {
        "location": f"{lat},{lng}",
        "radius": 40,
        "types" : "hotel",
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try: 
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
        
hotel_df["Hotel Name"] = hotellist
#hotel_df = hotel_df.dropna(how='any')
hotel_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
6,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,
1,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,CIDREIRA
5,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,Morondava
2,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,Shindholi
4,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Avenue Hassan I


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#narrowed_city_df=pd.DataFrame()
hotel_df=hotel_df.copy()
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [9]:
# Add marker layer ontop of heat map

# markers = gmaps.marker_layer(locations)
markers = gmaps.marker_layer(locations, info_box_content= [f"HOTEL NAME:  \n{hotel}"  for hotel in hotellist])

fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))